In [1]:
from ultralytics import YOLO
import cv2
import os
import numpy as np
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

model = YOLO("arkhyzS1000.pt")

In [2]:
def free_vram_func():
    # Create a PyTorch device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Get the free VRAM memory
    free_vram = torch.cuda.get_device_properties(device).total_memory - torch.cuda.memory_allocated(device)

    print("Free VRAM memory:", free_vram, "Allocated:", torch.cuda.memory_allocated(device))

In [3]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

In [4]:
image_folder = "../inference_data/ideal50"  # change this to your folder path
images = []
error_log = open("logs/files_error_log.txt", "a")  # open text file for writing
j=0
num_files = len([f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))])

print("Number of files in the folder:", num_files)

Number of files in the folder: 50


In [5]:
def prepare_files():
    global images
    global j
    error_status = False
    images = []
    j_max = j+25
    # get a sorted list of files in the image folder
    file_list = sorted(os.listdir(image_folder), key=lambda x: int(x.split("image")[1].split(".jpg")[0]))
    for file in file_list:
        if file.endswith(".jpg"):
            try:
                # extract the image number from the file
                image_num = int(file.split("image")[1].split(".jpg")[0])
                if j <= image_num <= j_max:
                    img_path = os.path.join(image_folder, file)
                    # read image with OpenCV
                    img = cv2.imread(img_path)
                    if img is None:
                        raise IOError  # raise an error if the image is not read successfully
                    # convert OpenCV image to numpy array
                    img_array = np.array(img)
                    # append to images list
                    print(f"append {image_num}")
                    images.append(img_array)
            except (IndexError, ValueError, IOError):
                error_status = True
                print(f"Unable to read image {file}")
                error_log.write(f"Unable to read image {file}\n")

    error_log.close()  # close text file when done
    j = j_max
    if not error_status:
        print("Preparation completed with no errors!")
    else:
        print("errors was saved to files_error_log.txt")

In [6]:
def predict_image():
    results = model.predict(source=images, stream=True, save=True, save_txt=True, save_conf=True, conf=0.5)
    error_log_predict = open("logs/prediction_error_log.txt", "a")
    for result in results:
        boxes = result.boxes.data
        if boxes.shape[0] == 0:
            error_log_predict.write(f"no detections :{j}:\n")
    error_log_predict.close()
    torch.cuda.empty_cache()

In [7]:
while j < num_files:
    prepare_files()
    model = YOLO("arkhyzS1000.pt")
    predict_image()
    free_gpu_cache()
# free_vram_func()

append 0
append 1
append 2
append 3
append 4
append 5
append 6
append 7
append 8
append 9
append 10
append 11
append 12
append 13
append 14
append 15
append 16
append 17
append 18
append 19
append 20
append 21
append 22
append 23
append 24
append 25
Preparation completed with no errors!



0: 1280x1280 2  1.5 s, 2  1.5 s, 6  1 s, 1: 1280x1280 2  1.5 s, 2  1.5 s, 2  5 s, 2: 1280x1280 1  1.5 , 4  5 s, 1  1 , 3: 1280x1280 1  0.5 , 2  1.5 s, 2  1.5 s, 2  1 s, 4: 1280x1280 3  0.5 s, 5: 1280x1280 5  1.5 s, 6: 1280x1280 5  5 s, 7: 1280x1280 3  0.5 s, 3  1.5 s, 8: 1280x1280 3  1 s, 9: 1280x1280 3  1.5 s, 10: 1280x1280 5  0.5 s, 8  1 s, 11: 1280x1280 4  0.5 s, 3  1.5 s, 12: 1280x1280 5  0.5 s, 13: 1280x1280 3  0.5 s, 2  1.5 s, 2  1.5 s, 2  5 s, 2  0,75   s, 14: 1280x1280 4  0.5 s, 15: 1280x1280 5  1.5 s, 16: 1280x1280 6  0.5 s, 17: 1280x1280 2  1.5 s, 3  1.5 s, 18: 1280x1280 (no detections), 19: 1280x1280 2  0.5 s, 3  5 s, 20: 1280x1280 3  0.5 s, 2  1.5 s, 21: 1280x1280 6  0.5 s, 6  1 s, 22: 1280x1280 8  0.5 s, 23: 1280x1280 5  0.5 s, 24: 1280x1280 3  1.5 s, 25: 1280x1280 6  0.5 s, 951.6ms
Speed: 10.2ms preprocess, 36.6ms inference, 1.8ms postprocess per image at shape (1, 3, 1280, 1280)
Results saved to C:\Users\1070PC\PycharmProjects\ultralytics\runs\detect\predict20
25 labels

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  1% | 18% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  4% | 10% |
append 25
append 26
append 27
append 28
append 29
append 30
append 31
append 32
append 33
append 34
append 35
append 36
append 37
append 38
append 39
append 40
append 41
append 42
append 43
append 44
append 45
append 46
append 47
append 48
append 49
Preparation completed with no errors!


RuntimeError: CUDA error: invalid argument
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# im2 = cv2.imread("../inference_data/img3.jpg")
# if im2 is not None:
#     print("Image exists.")
# else:
#     print("Image does not exist.")
# results = model.predict(source=im2, save=True, save_txt=True)

# results = model.predict(source="folder", show=True) # Display preds. Accepts all YOLO predict arguments
#
# im2 = cv2.imread("bus.jpg")
# results = model.predict(source=im2, save=True, save_txt=True)  # save predictions as labels
#
# # from list of PIL/ndarray
# results = model.predict(source=[im1, im2])
#
#
# results = model.predict(source=0, stream=True)
#
# for result in results:
#     # Detection
#     result.boxes.xyxy   # box with xyxy format, (N, 4)
#     result.boxes.xywh   # box with xywh format, (N, 4)
#     result.boxes.xyxyn  # box with xyxy format but normalized, (N, 4)
#     result.boxes.xywhn  # box with xywh format but normalized, (N, 4)
#     result.boxes.conf   # confidence score, (N, 1)
#     result.boxes.cls    # cls, (N, 1)
#
# # Each result is composed of torch.Tensor by default,
# # in which you can easily use following functionality:
# result = result.cuda()
# result = result.cpu()
# result = result.to("cpu")
# result = result.numpy()